In [2]:
# Standard python numerical analysis imports
import numpy as np
from scipy import signal
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import h5py

# Gravitational wave specific imports
from gwpy.timeseries import TimeSeries
from gwosc import datasets
from gwosc.datasets import event_gps
import readligo as rl  # LIGO-specific data reading module

# Configure matplotlib for inline plotting
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


Data Loading and Initial Exploration

In [3]:
# Get event information
eventlist = datasets.find_datasets(type='events')
chosen_event = 'GW150914'  
t0 = event_gps(chosen_event)
print(f"GPS time for {chosen_event}: {t0}")

# Load strain data using GWpy
detector = 'H1'  # or 'L1'
strain = TimeSeries.fetch_open_data(detector, t0-14, t0+14, cache=False)


# Sampling parameters
fs = int(strain.sample_rate.value)  # sampling rate
time = strain.times.value  # both detectors have same time vector
dt = time[1] - time[0]  # time sample interval


# Display basic data information
print(f'H1 strain: len={len(strain)}, min={strain.min():.2e}, max={strain.max():.2e}')

KeyboardInterrupt: 

Basic Data Visualization

In [ ]:
# Plot raw strain data around the event
tevent = 1126259462.422  # GW150914 GPS time
deltat = 5.  # seconds around the event

# Fetch L1 strain data as well
detector_L1 = 'L1'

strain_H1 = TimeSeries.fetch_open_data(detector, t0-14, t0+14, cache=False)
time_H1 = strain_H1.times.value

strain_L1 = TimeSeries.fetch_open_data(detector_L1, t0-14, t0+14, cache=False)
time_L1 = strain_L1.times.value


# Index for time interval for both H1 and L1
indxt_H1 = np.where((time_H1 >= tevent-deltat) & (time_H1 < tevent+deltat))
indxt_L1 = np.where((time_L1 >= tevent-deltat) & (time_L1 < tevent+deltat))


plt.figure(figsize=(12, 6))
plt.plot(time[indxt_H1]-tevent, strain[indxt_H1], '#1f77b4', linewidth=1, label='H1 strain')
plt.plot(time_L1[indxt_L1]-tevent, strain_L1[indxt_L1], '#ff7f0e', linewidth=1, label='L1 strain')
plt.xlabel(f'Time (s) since {tevent}')
plt.ylabel('Strain')
plt.legend()
plt.title('Advanced LIGO Strain Data Near GW150914')
plt.grid(True)
plt.show()

Power Spectral Density Analysis

In [ ]:
# Compute Power Spectral Densities (PSDs)
NFFT = 1*fs  # FFT length
fmin = 10
fmax = 2000

Pxx_H1, freqs = mlab.psd(strain_H1, Fs=fs, NFFT=NFFT)
Pxx_L1, freqs = mlab.psd(strain_L1, Fs=fs, NFFT=NFFT)

# Create interpolation functions for whitening
psd_H1 = interp1d(freqs, Pxx_H1)
psd_L1 = interp1d(freqs, Pxx_L1)

# Plot Amplitude Spectral Densities (ASDs)
plt.figure(figsize=(12, 8))
plt.loglog(freqs, np.sqrt(Pxx_H1), '#1f77b4', label='H1 strain')
plt.loglog(freqs, np.sqrt(Pxx_L1), '#ff7f0e', label='L1 strain')
plt.axis([fmin, fmax, 1e-24, 1e-19])
plt.grid(True)
plt.ylabel('ASD (strain/√Hz)')
plt.xlabel('Frequency (Hz)')
plt.legend()
plt.title('Advanced LIGO Amplitude Spectral Density')
plt.show()


Data Whitening Function

In [ ]:
def whiten(strain, interp_psd, dt):
    """
    Whiten strain data using the power spectral density
    """
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)

    # Transform to frequency domain, divide by ASD, transform back
    hf = np.fft.rfft(strain)
    white_hf = hf / (np.sqrt(interp_psd(freqs) / dt / 2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)

    return white_ht

# Apply whitening
strain_H1_whiten = whiten(strain_H1, psd_H1, dt)
strain_L1_whiten = whiten(strain_L1, psd_L1, dt)

# Bandpass filter to remove high frequency noise
bb, ab = butter(4, [20.*2./fs, 300.*2./fs], btype='band')
strain_H1_whitenbp = filtfilt(bb, ab, strain_H1_whiten)
strain_L1_whitenbp = filtfilt(bb, ab, strain_L1_whiten)


Signal Visualization After Processing

In [ ]:
# Plot whitened and bandpassed data
# Shift L1 by 7 ms and invert (detector geometry)
strain_L1_shift = -np.roll(strain_L1_whitenbp, int(0.007*fs))

plt.figure(figsize=(12, 8))
plt.plot(time-tevent, strain_H1_whitenbp, '#1f77b4', label='H1 strain')
plt.plot(time-tevent, strain_L1_shift, '#ff7f0e', label='L1 strain')
plt.xlim([-0.1, 0.05])
plt.ylim([-4, 4])
plt.xlabel(f'Time (s) since {tevent}')
plt.ylabel('Whitened strain')
plt.legend()
plt.title('Advanced LIGO Whitened Strain Data Near GW150914')
plt.grid(True)
plt.show()


Spectrogram Generation

In [ ]:
# Create spectrograms to visualize time-frequency content
deltat = 10.  # seconds around event
indxt = np.where((time_H1 >= tevent-deltat) & (time_H1 < tevent+deltat))

# Spectrogram parameters
NFFT = fs//16  # FFT length
NOVL = NFFT*15//16  # overlap
window = np.blackman(NFFT)
spec_cmap = 'viridis'

# H1 Spectrogram
plt.figure(figsize=(12, 8))
spec_H1, freqs, bins, im = plt.specgram(
    strain_H1_whiten[indxt],
    NFFT=NFFT,
    Fs=fs,
    window=window,
    noverlap=NOVL,
    cmap=spec_cmap,
    xextent=[-deltat, deltat]
)
plt.xlabel(f'Time (s) since {tevent}')
plt.ylabel('Frequency (Hz)')
plt.colorbar(label='Amplitude')
plt.axis([-0.5, 0.5, 0, 500])
plt.title('H1 Whitened Spectrogram Near GW150914')
plt.show()


Fiter design and Application




In [ ]:
def iir_bandstops(fstops, fs, order=4):
    """
    Design IIR bandstop (notch) filters for spectral line removal
    """
    nyq = 0.5 * fs
    zd = np.array([])
    pd = np.array([])
    kd = 1

    for fstopData in fstops:
        fstop = fstopData[0]
        df = fstopData[1]
        df2 = fstopData[2]

        low = (fstop - df) / nyq
        high = (fstop + df) / nyq
        low2 = (fstop - df2) / nyq
        high2 = (fstop + df2) / nyq

        z, p, k = iirdesign([low, high], [low2, high2],
                           gpass=1, gstop=6, ftype='ellip', output='zpk')
        zd = np.append(zd, z)
        pd = np.append(pd, p)

    b, a = zpk2tf(zd, pd, kd)
    return b, a

def get_filter_coefs(fs):
    """
    Get bandpass and notch filter coefficients
    """
    coefs = []

    # Bandpass filter parameters
    lowcut = 43
    highcut = 260
    order = 4

    nyq = 0.5*fs
    low = lowcut / nyq
    high = highcut / nyq
    bb, ab = butter(order, [low, high], btype='band')
    coefs.append((bb, ab))

    # Notch filters for known spectral lines
    notchesAbsolute = np.array([
        14.0, 34.70, 35.30, 35.90, 36.70, 37.30, 40.95, 60.00,
        120.00, 179.99, 304.99, 331.49, 510.02, 1009.99
    ])

    for notchf in notchesAbsolute:
        bn, an = iir_bandstops(np.array([[notchf, 1, 0.1]]), fs, order=4)
        coefs.append((bn, an))

    return coefs

def filter_data(data_in, coefs):
    """
    Apply series of filters to data
    """
    data = data_in.copy()
    for coef in coefs:
        b, a = coef
        data = filtfilt(b, a, data)
    return data


Advanced signal processing

In [ ]:
# Get filter coefficients and apply filtering
coefs = get_filter_coefs(fs)
strain_H1_filt = filter_data(strain_H1, coefs)
strain_L1_filt = filter_data(strain_L1, coefs)

# Plot filtered results
strain_L1_filt_shift = -np.roll(strain_L1_filt, int(0.007*fs))

plt.figure(figsize=(12, 8))
plt.plot(time-tevent, strain_H1_filt, '#ff7f0e', label='H1 strain')
plt.plot(time-tevent, strain_L1_filt_shift, '#1f77b4', label='L1 strain')
plt.xlim([-0.2, 0.1])
plt.xlabel(f'Time (s) since {tevent}')
plt.ylabel('Filtered strain')
plt.legend()
plt.title('Advanced LIGO Filtered Strain Data Near GW150914')
plt.grid(True)
plt.show()


Data Quality Assessment

In [ ]:
# Analyze data quality flags
print("Data Quality Information:")

# Check for data gaps
strain_data_H1 = strain
strain_data_L1 = strain_L1

print("\nH1 Data Quality:")
nan_count_H1 = np.sum(np.isnan(strain_data_H1))
print(f"  NaN values: {nan_count_H1}/{len(strain_data_H1)}")

print("\nL1 Data Quality:")
nan_count_L1 = np.sum(np.isnan(strain_data_L1))
print(f"  NaN values: {nan_count_L1}/{len(strain_data_L1)}")